<a href="https://colab.research.google.com/github/197315/Encuestas/blob/main/Encuestas_Adicion_de_variables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade gspread pandas gspread-dataframe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 66.5 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
  Attempting uninstall: gspread
    Found existing installation: gspread 6.0.2
    Uninstalling gspread-6.0.2:
      Successfully uninstalled gspread-6.0.2
  Attempting uninstall: gspread-dataframe
    Found existing installation: gspread-dataframe 3.3.1
    Uninstalling gspread-dataframe-3.3.1:
      Successfully uninstalled gspread-dataframe-3.3.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.10.1 requires pandas<2.2.3dev0,>=2.0, but you have pandas 2.2.3 which i

In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_columns', None)

from google.colab import auth
from google.auth import default
import gspread

# Autenticar cuenta de Google
auth.authenticate_user()

# Configurar la autorización
creds, _ = default()
gc = gspread.authorize(creds)

## Empresas Arg Master

In [2]:
PONDERADOR = "pondera.supervielle"
FILTRO_SEGMENTO = 'Seg.Supervielle'

Inputs

In [3]:
# Abre la hoja de cálculo usando la URL
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1k7gIC9LYp6cKtMYG3kwZoWmTBVyuT8uBjzQRVOboYs0/edit?gid=2141315370#gid=2141315370"
sheet = gc.open_by_url(spreadsheet_url)

# Selecciona una pestaña específica
worksheet = sheet.worksheet("spss_labels")  # Cambia "Hoja1" por el nombre de tu pestaña

# Convierte los datos a un DataFrame
data = worksheet.get_all_records()
catalog = pd.DataFrame(data).drop(columns='variable_raw')
catalog.head()

,variable,valor,Etiqueta,
0,FF1,1,PERSONA FISICA,
1,FF1,2,PERSONA JURÍDICA,
2,FF2,1,SI,
3,FF2,2,NO. Sólo utiliza bancos para su vida personal,
4,FF2,3,NO. No utilizo bancos,


In [4]:
csv_file = 'Empresas_Arg_Master'
master = pd.read_csv(csv_file+'.csv', sep=";")

In [5]:
base = pd.read_excel("Base Empresas 2024.xlsx")

In [ ]:
vars = base.columns

Procesamiento

In [ ]:
try:
  new_df = master[master.edicion >= 2020]
except:
  new_df = master[master.Edicion >= 2020]

In [ ]:
new_df.head()

,ResponseID,edicion,variable,valor
584033,133025176,2020,NPS.6,Santander
584034,133025176,2020,NPS.8,Bapro
584035,133025176,2020,NPS.6,Galicia
584036,133025176,2020,NPS.8,Nacion
584037,133025176,2020,NPS.6,Patagonia


### Variables de Financiamiento

In [ ]:
melt = pd.melt(base, id_vars=['ResponseID'],
                    value_vars=['B0_1', 'B4', 'B4_1_1', 'B4_1_2', 'B4_1_3', 'B4_1_4'],
                    var_name='variable', value_name='etiqueta')

In [ ]:
melt.tail()

,ResponseID,variable,etiqueta
8353,211042378,B4_1_4,NaN
8354,211055753,B4_1_4,NaN
8355,211056180,B4_1_4,NaN
8356,211091647,B4_1_4,NaN
8357,211096876,B4_1_4,NaN


In [ ]:
melt_tmp = melt.dropna(subset=['etiqueta'])
melt_tmp['etiqueta'] = melt_tmp['etiqueta'].astype('int')
melt_tmp['etiqueta'] = melt_tmp['etiqueta'].astype('object')
melt_tmp['edicion'] = 2024
resultado = melt_tmp[['ResponseID','edicion','variable','etiqueta']]

<ipython-input-321-f410e36c9991>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  melt_tmp['etiqueta'] = melt_tmp['etiqueta'].astype('int')
<ipython-input-321-f410e36c9991>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  melt_tmp['etiqueta'] = melt_tmp['etiqueta'].astype('object')
<ipython-input-321-f410e36c9991>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

In [ ]:
resultado_fin = (
    pd.merge(resultado,
             catalog,
             left_on=['variable','etiqueta'],
             right_on=['variable','valor'],
             how='left')
    .drop(columns=['etiqueta', 'valor', ''])
    .rename(columns = {'Etiqueta': 'valor'})
    )

In [ ]:
resultado_fin.tail()

,ResponseID,edicion,variable,valor
1880,204768512,2024,B4_1_4,Compra/renovación de producto mobiliarios (maq...
1881,206912844,2024,B4_1_4,Compra/renovación de producto mobiliarios (maq...
1882,207475040,2024,B4_1_4,Compra/renovación de producto mobiliarios (maq...
1883,209497350,2024,B4_1_4,Compra/renovación de producto mobiliarios (maq...
1884,210795732,2024,B4_1_4,Compra/renovación de producto mobiliarios (maq...


### Variables Responsabilidad Social A6 - A7

### Responsabilidad Social (A6 - A7 - A8)

In [ ]:
resp_a6_melt = pd.melt(base, id_vars=['ResponseID'],
                    value_vars='A6',
                    var_name='vars', value_name='etiqueta')

In [ ]:
resp_melt_tmp = resp_a6_melt.dropna(subset=['etiqueta'])
resp_melt_tmp['etiqueta'] = resp_melt_tmp['etiqueta'].astype('int')
resp_melt_tmp['etiqueta'] = resp_melt_tmp['etiqueta'].astype('object')
resp_melt_tmp['edicion'] = 2024
resultado_a6 = resp_melt_tmp[['ResponseID','edicion','vars','etiqueta']]

<ipython-input-325-4e371a175e14>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resp_melt_tmp['etiqueta'] = resp_melt_tmp['etiqueta'].astype('int')
<ipython-input-325-4e371a175e14>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resp_melt_tmp['etiqueta'] = resp_melt_tmp['etiqueta'].astype('object')
<ipython-input-325-4e371a175e14>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [ ]:
resultado_a6_vf = (
    pd.merge(resultado_a6,
             catalog,
             left_on=['vars','etiqueta'],
             right_on=['variable','valor'],
             how='left')
    .drop(columns=['etiqueta', 'valor', ''])
    .rename(columns = {'Etiqueta': 'valor'})
    )
resultado_a6_vf['variable'] = 'Responsabilidad social corporativa'
resultado_a6_vf = resultado_a6_vf[resultado_a6_vf.valor != 'Si'][['ResponseID','edicion','variable','valor']]

#### A7

In [ ]:
resp_soc_vars = ['A7_i_1', 'A7_i_2', 'A7_i_3', 'A7_i_4', 'A7_i_5', 'A7_i_6', 'A7_i_7', 'A7_i_8',
                 'A7_i_9', 'A7_i_10', 'A7_i_13', 'A7_i_14', 'A7_i_15', 'A7_i_16', 'A7_i_17',
                 'A7_i_20', 'A7_i_23', 'A7_i_24', 'A7_i_25', 'A7_i_26', 'A7_i_27', 'A7_i_28',
                 'A7_i_29','A7_i_30', 'A7_i_31', 'A7_i_32', 'A7_i_33', 'A7_i_35', 'A7_i_37',
                 'A7_i_39', 'A7_i_40', 'A7_i_50', 'A7_i_51', 'A7_i_57', 'A7_i_74', 'A7_i_75',
                 'A7_i_101', 'A7_i_100', 'A7_i_102', 'A7_i_105', 'A7_i_103', 'A7_i_127',
                 'A7_i_131', 'A7_i_129', 'A7_i_134', 'A7_i_135', 'A7_i_136', 'A7_i_90',
                 'A7_i_91', 'A7_i_191']

In [ ]:
resp_a7_melt = pd.melt(base, id_vars=['ResponseID'],
                    value_vars=resp_soc_vars,
                    var_name='vars', value_name='etiqueta')

In [ ]:
resultado_a7 = resp_a7_melt.dropna(subset=['etiqueta'])
resultado_a7['etiqueta'] = resultado_a7['etiqueta'].astype('int')
resultado_a7['etiqueta'] = resultado_a7['etiqueta'].astype('object')
resultado_a7['edicion'] = 2024
resultado_a7 = resultado_a7[['ResponseID','edicion','vars','etiqueta']]

<ipython-input-329-1ca92383ab45>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultado_a7['etiqueta'] = resultado_a7['etiqueta'].astype('int')
<ipython-input-329-1ca92383ab45>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultado_a7['etiqueta'] = resultado_a7['etiqueta'].astype('object')
<ipython-input-329-1ca92383ab45>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [ ]:
resultado_a7

,ResponseID,edicion,vars,etiqueta
2,206321657,2024,A7_i_1,1
6,205135188,2024,A7_i_1,1
11,203238617,2024,A7_i_1,1
23,207895284,2024,A7_i_1,1
31,203050262,2024,A7_i_1,1
...,...,...,...,...
52111,202398580,2024,A7_i_100,100
52217,202413690,2024,A7_i_100,100
55500,206337248,2024,A7_i_105,105
56893,206337248,2024,A7_i_103,103


In [ ]:
resultado_a7_vf = (
    pd.merge(resultado_a7,
             catalog,
             left_on=['vars','etiqueta'],
             right_on=['variable','valor'],
             how='left')
    .drop(columns=['etiqueta', 'valor', ''])
    .rename(columns = {'Etiqueta': 'valor'})
    )
resultado_a7_vf['variable'] = 'Responsabilidad social corporativa'
resultado_a7_vf = resultado_a7_vf[['ResponseID','edicion','variable','valor']]

In [ ]:
resultado_a7_vf

,ResponseID,edicion,variable,valor
0,206321657,2024,Responsabilidad social corporativa,Santander
1,205135188,2024,Responsabilidad social corporativa,Santander
2,203238617,2024,Responsabilidad social corporativa,Santander
3,207895284,2024,Responsabilidad social corporativa,Santander
4,203050262,2024,Responsabilidad social corporativa,Santander
...,...,...,...,...
389,202398580,2024,Responsabilidad social corporativa,MercadoPago
390,202413690,2024,Responsabilidad social corporativa,MercadoPago
391,206337248,2024,Responsabilidad social corporativa,"MODO ( Instalada en forma independiente, no de..."
392,206337248,2024,Responsabilidad social corporativa,Cuenta DNI


#### A8

In [ ]:
resp_a8_melt = pd.melt(base, id_vars=['ResponseID'],
                    value_vars='A8',
                    var_name='tmp', value_name='etiqueta')

In [ ]:
resp_melt_tmp = resp_a8_melt.dropna(subset=['etiqueta'])
resp_melt_tmp['etiqueta'] = resp_melt_tmp['etiqueta'].astype('int')
resp_melt_tmp['etiqueta'] = resp_melt_tmp['etiqueta'].astype('object')
resp_melt_tmp['edicion'] = 2024

<ipython-input-334-9bace2c0c66b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resp_melt_tmp['etiqueta'] = resp_melt_tmp['etiqueta'].astype('int')
<ipython-input-334-9bace2c0c66b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resp_melt_tmp['etiqueta'] = resp_melt_tmp['etiqueta'].astype('object')
<ipython-input-334-9bace2c0c66b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [ ]:
resp_melt_tmp['var'] = 'ResponSocial'
resp_melt_tmp['valor'] = 'NoCorresponde'

<ipython-input-335-f9c81f9824db>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resp_melt_tmp['var'] = 'ResponSocial'
<ipython-input-335-f9c81f9824db>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resp_melt_tmp['valor'] = 'NoCorresponde'


In [ ]:
resp_melt_tmp['variable'] = resp_melt_tmp['var'].astype(str) + "-" + resp_melt_tmp['etiqueta'].astype(str)
resp_melt_tmp['variable'].replace({'ResponSocial-99': 'ResponSocial-No sabe'}, inplace= True)

<ipython-input-336-7b70540e1bd6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resp_melt_tmp['variable'] = resp_melt_tmp['var'].astype(str) + "-" + resp_melt_tmp['etiqueta'].astype(str)
<ipython-input-336-7b70540e1bd6>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  resp_melt_tmp['va

In [ ]:
resp_melt_tmp = resp_melt_tmp[['ResponseID','edicion','variable','valor']]

In [ ]:
resp_melt_tmp.head()

,ResponseID,edicion,variable,valor
0,208174782,2024,ResponSocial-9,NoCorresponde
1,208851395,2024,ResponSocial-No sabe,NoCorresponde
2,206321657,2024,ResponSocial-9,NoCorresponde
3,202308487,2024,ResponSocial-8,NoCorresponde
4,208102295,2024,ResponSocial-8,NoCorresponde


## Union

In [ ]:
new_master = pd.concat([new_df, resultado_fin], ignore_index=True)

In [ ]:
new_master = pd.concat([new_master, resultado_a6_vf], ignore_index=True)

In [ ]:
new_master = pd.concat([new_master, resultado_a7_vf], ignore_index=True)

In [ ]:
new_master = pd.concat([new_master, resp_melt_tmp], ignore_index=True)

In [ ]:
new_master.head()

,ResponseID,edicion,variable,valor
0,133025176,2020,NPS.6,Santander
1,133025176,2020,NPS.8,Bapro
2,133025176,2020,NPS.6,Galicia
3,133025176,2020,NPS.8,Nacion
4,133025176,2020,NPS.6,Patagonia


In [ ]:
new_master.tail()

,ResponseID,edicion,variable,valor
970159,210997157,2024,ResponSocial-8,NoCorresponde
970160,210998667,2024,ResponSocial-No sabe,NoCorresponde
970161,211008410,2024,ResponSocial-8,NoCorresponde
970162,211010732,2024,ResponSocial-9,NoCorresponde
970163,211091647,2024,ResponSocial-7,NoCorresponde


In [ ]:
new_master.to_csv(csv_file+'2.csv', sep=";", index=False)

# Recorte de Años de CSVs

In [ ]:
csv_file = 'Empresas_Arg_Master'

In [ ]:
try:
  df = pd.read_csv(csv_file+'.csv', sep=";", encoding="latin1")
except:
  df = pd.read_csv(csv_file+'.csv', sep=";")

In [ ]:
df.dtypes

,0
ResponseID,int64
edicion,int64
variable,object
valor,object


In [ ]:
df.head()

,ResponseID,edicion,variable,valor
0,133025176,2020,NPS.6,Santander
1,133025176,2020,NPS.8,Bapro
2,133025176,2020,NPS.6,Galicia
3,133025176,2020,NPS.8,Nacion
4,133025176,2020,NPS.6,Patagonia


In [ ]:
try:
  new_df = df[df.edicion >= 2020]
except:
  new_df = df[df.Edicion >= 2020]

In [ ]:
new_df.head()

,ResponseID,edicion,actividad,region,origen.capital,tipo.base,CUIT,Nombre,Facturacion
9941,134729880,2020,Comercio,Interior,Nacional,OM,NaN,NaN,NaN
9942,133594489,2020,Comercio,Interior,Nacional,OM,NaN,NaN,NaN
9943,135887878,2020,Servicios,Interior,Nacional,OM,NaN,NaN,NaN
9944,135207583,2020,Industria,Interior,Nacional,OM,NaN,NaN,NaN
9945,135147556,2020,Comercio,Interior,Nacional,OM,NaN,NaN,NaN


In [ ]:
try:
  print(new_df.Edicion.unique())
except:
  print(new_df.edicion.unique())

[2020 2021 2022 2023 2024]


In [ ]:
new_df.to_csv(csv_file+'2.csv', sep=";", index=False)

# Agregar variables a CSV